# Convolutional Neural Network

### Importing the libraries

In [23]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [24]:
tf.__version__

'2.1.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [25]:
# Increasing the images variety = Image Augmentation = In order not to let Over-learning happen! = from "https://keras.io/api/preprocessing/image/" datagenerator section!

train_datagen = ImageDataGenerator(
        rescale = 1./255, #on each pixel!
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)

training_set = train_datagen.flow_from_directory(
        'dataset/training_set/',
        target_size=(64, 64), #target_size=(150, 150): by experience this took too long to train. So we choose (64, 64).
        batch_size=32,
        class_mode='binary') #"cat / dog" -> so "binary".

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [26]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64), #should be the same as "training_set" above.
        batch_size=32, #~
        class_mode='binary') #~

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [27]:
from tensorflow.keras.models import Sequential

cnn = Sequential()

### Step 1 - Convolution

In [28]:
from tensorflow.keras.layers import Conv2D

cnn.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = (64, 64, 3)))
#A famous and classic CNN architecture has "filters = 32" for the first convolutional layer. And then a "filters = 32" for the second Convolutional layer!
#input_shape = (64, 64, 3): Since we have chosen "target_size=(64, 64)" above, we have to specify the input shape as I did. And if the images were black and white, then "input_shape = (64, 64, 1)".

### Step 2 - Pooling

In [29]:
from tensorflow.keras.layers import MaxPool2D

cnn.add(MaxPool2D(pool_size = (2, 2), strides = 2, padding = 'valid'))
#pool_size=(2, 2): recomended to choose (2, 2)! (or just " = 2")
#strides = 2: recomended to choose 2!

### Adding a second convolutional layer

In [30]:
cnn.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu')) #no need to specify "input_shape". Since it is only used for the first layer.
cnn.add(MaxPool2D(pool_size = (2, 2), strides = 2, padding = 'valid'))

### Step 3 - Flattening

In [31]:
from tensorflow.keras.layers import Flatten

cnn.add(Flatten()) #no hyperparameters!

### Step 4 - Full Connection

In [32]:
from tensorflow.keras.layers import Dense

cnn.add(Dense(units = 128, activation = 'relu'))
#As far as you have NOT reached the output layer, it is recomended to choses "activation = 'relu'", which is the rect function.

### Step 5 - Output Layer

In [33]:
cnn.add(Dense(units = 1, activation = 'sigmoid'))
#units = 1: since we have binary classification (Cat / dog).
#activation = 'sigmoid': Since we have binary classification (dog / cat). If it was multiple classification, we HAD TO use "softmax".

## Part 3 - Training the CNN

### Compiling the CNN

In [34]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#loss = 'binary_crossentropy': since we have binary classification (Cat / dog). If it was multiple classification, we HAD TO use "categorical_crossentropy".

### Training the CNN on the Training set and evaluating it on the Test set

In [35]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
#epochs = 25: should be an amount which finalyy converges.

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 63 steps
Epoch 1/25
250/250 [==============================] - 157s 629ms/step - loss: 0.6869 - accuracy: 0.5584 - val_loss: 0.6464 - val_accuracy: 0.6415
Epoch 2/25
250/250 [==============================] - 81s 323ms/step - loss: 0.6315 - accuracy: 0.6524 - val_loss: 0.6296 - val_accuracy: 0.6540
Epoch 3/25
250/250 [==============================] - 63s 252ms/step - loss: 0.5849 - accuracy: 0.6895 - val_loss: 0.5604 - val_accuracy: 0.7095
Epoch 4/25
250/250 [==============================] - 73s 291ms/step - loss: 0.5449 - accuracy: 0.7225 - val_loss: 0.5256 - val_accuracy: 0.7435
Epoch 5/25
250/250 [==============================] - 67s 267ms/step - loss: 0.5220 - accuracy: 0.7406 - val_loss: 0.5107 - val_accuracy: 0.7490
Epoch 6/25
250/250 [==============================] - 70s 281ms/step - loss: 0.4895 - accuracy: 0.7584 - val_loss: 0.4938 - val_accuracy: 0.7615
Epoch 7/25
250/250 [================

## Part 4 - Making a single prediction

In [37]:
import numpy as np
from keras.preprocessing.image import load_img #all from "https://keras.io/api/preprocessing/image/"
from keras.preprocessing.image import img_to_array

single_test_image = load_img(path = 'dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
single_test_image = img_to_array(single_test_image) #converts an image to a numpy format!
single_test_image = np.expand_dims(single_test_image, axis = 0) #because we used "batch" ("batch_size = 32"), then we HAVE TO reshape it in a way that an extra dimension for the batch size is added to the "single_test_image". And "axis = 0" means that this new dimension is the 0 or the first dimension.

result = cnn.predict(single_test_image)

#encoding the output: (dog:1, cat:0)
training_set.class_indices #!!!

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [38]:
print(prediction)

dog


In [ ]:
#So it predicted the 'dog' correctly!! The total accuracy as read by the last epoch is "90%"!

In [39]:
#Now, a random dog image from the test set, for instance: 4689. So we have to copy and paste this image in the (dataset/single_prediction) folder.

single_test_image = load_img(path = 'dataset/single_prediction/dog.4689.jpg', target_size = (64, 64))
single_test_image = img_to_array(single_test_image)
single_test_image = np.expand_dims(single_test_image, axis = 0)

result = cnn.predict(single_test_image)

training_set.class_indices #!!!

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [40]:
print(prediction)

dog


In [ ]:
#So it predicted correctly again!

In [45]:
#Now, a random cat image from the test set, for instance: 4538. So we have to copy and paste this image in the (dataset/single_prediction) folder.

single_test_image = load_img(path = 'dataset/single_prediction/cat.4538.jpg', target_size = (64, 64))
single_test_image = img_to_array(single_test_image)
single_test_image = np.expand_dims(single_test_image, axis = 0)

result = cnn.predict(single_test_image)

training_set.class_indices

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [46]:
print(prediction)

dog


In [ ]:
#Unfortunately this time the model could not preduct correctly!